In [4]:
# Load csv into dataframe
import pandas as pd
import numpy as np
import nltk
import string
import ast
import re
import unidecode
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter

# Columns with irrelevant data were deleted in Excel prior
df = pd.read_csv('./recipes.csv', index_col='RecipeId')



# Drop recipes who have a rating of 4 or less OR 3 or less total reviews
df = df.dropna()
df['AggregatedRating'] = df['AggregatedRating'].astype(float)
df['ReviewCount'] = df['ReviewCount'].astype(float)
df = df[df['AggregatedRating'] > 4.0]
df = df[df['ReviewCount'] > 3]

# Renumbered recipe ID's to match index
new_index_values = range(0, len(df))
df.index = new_index_values
df.index.name = 'RecipeID'

# Cleaned up ingredients and instructions column
df['RecipeIngredientParts'] = df['RecipeIngredientParts'].apply(lambda x: x.replace('c(','').replace(')','').replace('"','').replace('", "',' ').replace(',',''))
df['RecipeInstructions'] = df['RecipeInstructions'].apply(lambda x: x.replace('c(','').replace(')',''))
df['RecipeIngredientParts'] = df['RecipeIngredientParts'].str.split()



C:\Users\gazaw\AppData\Local\Temp\ipykernel_7184\1456223510.py:14: DtypeWarning: Columns (0,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./recipes.csv', index_col='RecipeId')


In [7]:
# Visualize the top 200 most common ingredients to remove common household ingredients (e.g. salt, pepper, oil, etc)
vocab = nltk.FreqDist()

for ingredients in df['RecipeIngredientParts']:
    
    vocab.update(ingredients)
    
for word, frequency in vocab.most_common(200):
    print(f'{word};{frequency}')


salt;36647
sugar;34975
pepper;30375
butter;27682
cheese;25781
garlic;24141
fresh;23407
flour;22066
onion;20681
ground;20089
powder;17617
water;16733
oil;15768
milk;14227
olive;14127
cream;12708
baking;12541
sauce;12524
eggs;12256
chicken;12161
black;11315
green;10944
vanilla;10848
juice;10615
lemon;10259
brown;9766
cloves;9730
red;9311
dried;9207
tomatoes;8946
white;8208
cinnamon;7748
parsley;7385
all-purpose;6870
egg;6693
onions;6372
vinegar;6244
beef;6081
parmesan;5972
broth;5719
soda;5603
mustard;5381
sour;5363
cheddar;5345
margarine;5237
wine;5190
basil;4726
tomato;4696
potatoes;4645
celery;4545
oregano;4388
boneless;4380
dry;4377
ginger;4364
soy;4356
cumin;4184
extract;4062
bell;3987
rice;3895
beans;3832
chili;3824
mayonnaise;3725
clove;3612
thyme;3541
skinless;3505
leaves;3484
paprika;3484
lime;3475
honey;3470
cayenne;3353
nutmeg;3152
unsalted;3151
cilantro;3055
bacon;3030
breasts;2986
corn;2941
Worcestershire;2920
cornstarch;2740
flakes;2620
light;2604
frozen;2574
of;2514
whole;

In [8]:
def ingredient_cleanup(ingredientList):
    
    terms_to_remove = ['salt', 'pepper', 'oil','of', 'zest','fresh', 'butter', 'sugar', 'water', 'cloves', 'all','purpose', 'black', 'pepper', 'garlic', 'powder', 'cinnamon', 'juice', 'paprika','clove', 'chili','ground','extra','virgin','granulated','cumin','dried','oregano', 'kosher', 'boneless','skinless','canola', 'basil', 'white','thyme','cilantro','vinegar','powder','confectioners','seasoning','lean','curry','light','crushed','dry','boiling','sea', 'plain']
    
    # turn ingredients from objects into a list
    if isinstance(ingredientList, list):
        ingredients = ingredientList
    else:
        ingredients = ast.literal_eval(ingredientList)
    
    # initialize lemmatizer and ingredient list to be returned
    lemmatizer = WordNetLemmatizer()
    cleaned_ingredients = []
    
    for i in ingredients:
        # Split words according to spaces and hyphens
        items = re.split(' |-', i)
        
        # Remove any non-alphabetic terms, set it lower-case and remove any accents
        items = [word for word in items if word.isalpha()]
        items = [word.lower() for word in items]
        items = [unidecode.unidecode(word) for word in items]
    
        # Lemmatize each word for analysis and comparison
        items =[lemmatizer.lemmatize(word) for word in items]
        # remove common terms ingredients listed in terms_to_remove
        items = [word for word in items if word not in terms_to_remove]
        
        if items:
            cleaned_ingredients.append(' '.join(items))
    cleaned_ingredients_str = ' '.join(cleaned_ingredients)
    return cleaned_ingredients_str

In [9]:
# Run lambda to apply function to database and display results
df['RecipeIngredientParts'] = df['RecipeIngredientParts'].apply(lambda x: ingredient_cleanup(x))

df.head()

,Name,RecipeIngredientParts,AggregatedRating,ReviewCount,RecipeInstructions
RecipeID,,,,,
0,Low-Fat Berry Blue Frozen Dessert,blueberry vanilla yogurt lemon,4.5,4.0,"""Toss 2 cups berries with sugar."", ""Let stand ..."
1,Best Lemonade,lemon rind lemon lemon,4.5,10.0,"""Into a 1 quart Jar with tight fitting lid, pu..."
2,Cabbage Soup,tomato cabbage onion carrot celery,4.5,11.0,"""Mix everything together and bring to a boil.""..."
3,Warm Chicken A La King,chicken flour milk celery button mushroom gree...,5.0,23.0,"""Melt 1 1/2 ozs butter, add the flour and cook..."
4,Chicken Breasts Lombardi,mushroom chicken breast half flour marsala chi...,5.0,21.0,"""Cook mushrooms in 2 tbsp butter in a large s..."


In [12]:
# Save dataframe to new CSV file to function as database
file_path = 'C:/Users/gazaw/Desktop/WGUCapstone/DataModel/recipes_parsed.csv'
recipes_parsed = df.to_csv(file_path, index=True)